In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [4]:
#ABILoading().get_abi_by_filename(abi_path)

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3])  
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,962,048 tx:0x0474fdf61a9bb9ee59c94d805493fe40cae4ef12e7a01fc9e48b3dc45cac9252
Swap at block:60,962,048 tx:0x02854c6119f40187e0ae7a951919d8a04c045affdc8531714742257cf004ead2
Swap at block:60,962,049 tx:0x1a41331ef9fa4aff62775216949d06ab96baa2859ec96306bae119b76c4ffd67
Swap at block:60,962,051 tx:0x453e57805efcf9bac5a9d209b2e0003b12aa269203c3d278cc09c955fd263fd9
Swap at block:60,962,051 tx:0x6e155cecc6dddc50b04ce3003f1e411849cfccc6c6b84ef7e30f9b6010ee8d56
Swap at block:60,962,051 tx:0x6e155cecc6dddc50b04ce3003f1e411849cfccc6c6b84ef7e30f9b6010ee8d56
Swap at block:60,962,052 tx:0xb8f4c5a19da698560cc27052813a4178f36e57c721bbae51bbb1ca549b71f029
Swap at block:60,962,052 tx:0x8f3e67cdccf91d75e3ec4bc6e80753e534785ca6fe982965c2dd2ccea33824b1
Swap at block:60,962,052 tx:0x8f3e67cdccf91d75e3ec4bc6e80753e534785ca6fe982965c2dd2ccea33824b1
Swap at block:60,962,052 tx:0x8f3e67cdccf91d75e3ec4bc6e80753e534785ca6fe982965c2dd2ccea33824b1
Swap at block:60,962,052 tx:0x8f3e67cdccf91d75e3ec

In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,swap,sushi,0x232eb60e4ea68a164988329e16305bcacccf2a97,0x0474fdf61a9bb9ee59c94d805493fe40cae4ef12e7a0...,60962048,1724460772,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv2pair,swap,sushi,0x8b4e00810c927bb1c02dee73d714a31121689ab3,0x02854c6119f40187e0ae7a951919d8a04c045affdc85...,60962048,1724460772,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv2pair,swap,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0x1a41331ef9fa4aff62775216949d06ab96baa2859ec9...,60962049,1724460776,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv2pair,swap,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0x453e57805efcf9bac5a9d209b2e0003b12aa269203c3...,60962051,1724460784,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv2pair,swap,sushi,0xeff9df0b108bd35bfa7ecb48d893e86f79c9563f,0x6e155cecc6dddc50b04ce3003f1e411849cfccc6c6b8...,60962051,1724460784,{'web3_type': <class 'web3._utils.datatypes.Sw...
5,polygon,uniswapv2pair,swap,sushi,0xadbf1854e5883eb8aa7baf50705338739e558e5b,0x6e155cecc6dddc50b04ce3003f1e411849cfccc6c6b8...,60962051,1724460784,{'web3_type': <class 'web3._utils.datatypes.Sw...
6,polygon,uniswapv2pair,swap,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xb8f4c5a19da698560cc27052813a4178f36e57c721bb...,60962052,1724460788,{'web3_type': <class 'web3._utils.datatypes.Sw...
7,polygon,uniswapv2pair,swap,sushi,0xadbf1854e5883eb8aa7baf50705338739e558e5b,0x8f3e67cdccf91d75e3ec4bc6e80753e534785ca6fe98...,60962052,1724460788,{'web3_type': <class 'web3._utils.datatypes.Sw...
8,polygon,uniswapv2pair,swap,sushi,0xadbf1854e5883eb8aa7baf50705338739e558e5b,0x8f3e67cdccf91d75e3ec4bc6e80753e534785ca6fe98...,60962052,1724460788,{'web3_type': <class 'web3._utils.datatypes.Sw...
9,polygon,uniswapv2pair,swap,sushi,0x604229c960e5cacf2aaeac8be68ac07ba9df81c3,0x8f3e67cdccf91d75e3ec4bc6e80753e534785ca6fe98...,60962052,1724460788,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0x232eb60e4ea68a164988329e16305bcacccf2a97',
  'tx_hash': '0x0474fdf61a9bb9ee59c94d805493fe40cae4ef12e7a01fc9e48b3dc45cac9252',
  'blk_num': 60962048,
  'timestamp': 1724460772,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xedf6066a2b290C185783862C7F4776A2C8077AD1',
   'token1': '0xdC6A0173c4c6198b47F16081C631Be0683F82D6D',
   'amount0In': 524393166102862364672,
   'amount1Out': 18460674}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0x8b4e00810c927bb1c02dee73d714a31121689ab3',
  'tx_hash': '0x02854c6119f40187e0ae7a951919d8a04c045affdc8531714742257cf004ead2',
  'blk_num': 60962048,
  'timestamp': 1724460772,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xDEF171Fe48CF0115B1d80b88dc8eAB59176FEe57',
   'token1': '0xDEF171Fe48CF0115B1d80b88dc8eAB59176FEe57',
   'amou